In [1]:
import pandas as pd 
import numpy as np 
import json 
import os

from tqdm import tqdm 

In [26]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)
np.set_printoptions(suppress=True)

In [16]:
#file_loc = "/Users/benji/Downloads/Kickstarter_2021-04-15T03_20_08_451Z/"
#file_loc = "/Users/benji/Downloads/Kickstarter_2021-03-18T03_20_11_507Z/"
file_loc = "/Users/benji/Downloads/Kickstarter_2021-02-11T03_20_07_976Z/"
file_loc = "/Users/benji/Downloads/Kickstarter_2020-04-16T03_20_04_541Z/"
files = sorted(os.listdir(file_loc))

In [17]:
def process_df(path_to_file):
    """
    Note: main speed limitation is json.loads() which is ~ slow 
    Averages around 1.65 seconds per iteration w/ a dataframe ~ 3k rows 
    """
    
    # load df 
    df = pd.read_csv(path_to_file)
    # filter rows to finished projects  
    df = df[df['state'].isin(['successful', 'failed'])]
    # add some features 
    df = df.assign(
        usd_goal = lambda x:x['goal'] * x['fx_rate'], 
        available_time = lambda x:x['deadline'] - x['launched_at'], # figure out how time is encoded?? 
        blurb_len = lambda x:x['blurb'].str.len()
    )
    # split up category 
    cat_cols_to_keep = ["id", "position", "parent_id", "color"]
    tmp = df['category'].apply(json.loads).apply(pd.Series)[cat_cols_to_keep]
    tmp.columns = "cat_" + tmp.columns 
    cat_cols_to_keep = ["cat_" + w for w in cat_cols_to_keep]
    df = pd.concat((df, tmp), axis = 1)
    # split up location 
    loc_cols_to_keep = ["id", "type", "state"]
    tmp = df['location'].fillna('{}').apply(json.loads).apply(pd.Series)[loc_cols_to_keep]
    tmp.columns = "loc_" + tmp.columns 
    loc_cols_to_keep = ["loc_" + w for w in loc_cols_to_keep]
    df = pd.concat((df, tmp), axis=1)
    #### 
    df_cols_to_keep = [
         "state", "usd_goal", "available_time", "blurb_len", "launched_at", "deadline", "blurb", 
        "name", "currency", "country", "is_starred", "is_starrable", "spotlight", "staff_pick", "photo"
    ]
    return df[df_cols_to_keep + cat_cols_to_keep + loc_cols_to_keep]

In [18]:
dfs = []
for f in tqdm(files):
    dfs.append( process_df(file_loc + f) )

100%|██████████| 56/56 [06:41<00:00,  7.16s/it]


In [19]:
dfs = pd.concat(dfs)

In [20]:
dfs.shape

(191849, 22)

In [21]:
dfs.to_csv("2020_04_processed_df.csv")

In [40]:
#### --> DROP DUPLICATE PROJECTS HERE <--- #### 
# note: it seems that the dataframes are cumulative, thus we need to identify duplicate projects and remove them. 
# To identify a duplicate project, I'm checking the: 
#        blurb and name, funding goal, date of launch, and deadline
# Hopefully, this should be sufficient to catch any duplicate projects 

main_file = "2021_04_processed_df.csv"
processed_files = [
    "2020_04_processed_df.csv", "2020_12_processed_df.csv", "2021_03_processed_df.csv", "2021_02_processed_df.csv"
]
main = pd.read_csv(main_file).drop(columns=["Unnamed: 0"])
print("Initial file:", main.shape)
for f in processed_files: 
    dat = pd.read_csv(f).drop(columns=["Unnamed: 0"])
    # combine and drop duplicates 
    tmp = pd.concat((dat, main)).drop_duplicates(subset=["blurb", "name", "usd_goal", "launched_at", "deadline"])
    main = tmp
    del dat, tmp # let's keep our memory as low as possible as we go 
    print("Adding", f, main.shape)
main.to_csv("all_processed_df.csv")

(208508, 22)
(249102, 22)
(309354, 22)
(363119, 22)
(422842, 22)
